<a href="https://colab.research.google.com/github/limiz6383/Project-AEI-/blob/main/movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets loralib sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.5 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]==4.33.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.4 MB/s eta 0:00:00


In [3]:
!pip install accelerate>=0.12.0

In [8]:
import pandas as pd
from google.colab import files
import io

In [33]:
url = "https://raw.githubusercontent.com/limiz6383/Project-AEI-/main/movie_lines.tsv?token=GHSAT0AAAAAACGPIIGU6DHUYKK4SK7MLHNMZI3JSTA"
df = pd.read_csv(url, sep = "\t", on_bad_lines="skip")
# uploaded = files.upload()
# file_path = io.BytesIO(uploaded['/content/movie_lines.tsv'])
# df = pd.read_csv(file_path, sep='\t', header=0, error_bad_lines=False)

In [46]:
experiment = df.copy()
new = experiment.iloc[:, 3:]
new.columns = ["Name", "Line"]
new['with_question_mark'] = new['Line'].apply(lambda x: x if pd.notna(x) and '?' in x else None)
new['without_question_mark'] = new['Line'].apply(lambda x: x if pd.notna(x) and '?' not in x else None)

new

<ipython-input-46-dd7321c8652a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['with_question_mark'] = new['Line'].apply(lambda x: x if pd.notna(x) and '?' in x else None)
<ipython-input-46-dd7321c8652a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['without_question_mark'] = new['Line'].apply(lambda x: x if pd.notna(x) and '?' not in x else None)


,Name,Line,with_question_mark,without_question_mark
0,CAMERON,They do to!,None,They do to!
1,BIANCA,I hope so.,None,I hope so.
2,CAMERON,She okay?,She okay?,None
3,BIANCA,Let's go.,None,Let's go.
4,CAMERON,Wow,None,Wow
...,...,...,...,...
293196,DURNFORD,Lord Chelmsford seems to want me to stay back ...,None,Lord Chelmsford seems to want me to stay back ...
293197,VEREKER,I'm to take the Sikali with the main column to...,None,I'm to take the Sikali with the main column to...
293198,DURNFORD,Your orders Mr Vereker?,Your orders Mr Vereker?,None
293199,DURNFORD,Good ones yes Mr Vereker. Gentlemen who can ri...,None,Good ones yes Mr Vereker. Gentlemen who can ri...


In [47]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new, test_size=0.2)

In [42]:
#preprocess data
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b",
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16)